In [1]:
# import libraries
import pandas as pd
import numpy as np  
from scraper import load_chrome_driver,get_distillation_temps

# set driver module
driver = load_chrome_driver()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\adams\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache
 


In [5]:

# retrieve the oil profile function given a temperature table, the function is a 3rd degree polynomial
def get_oil_profile_func(temp_tbl):
    return np.poly1d(np.polyfit(temp_tbl.index, temp_tbl.values, 3))


# creates an oil mixture profile function given 2 different oil lists 
def mixed_oil_distillation(oil_temps_a, oil_temps_b):
    percent_yield = [5,10,20,30,40,50,60,70,80,90,95,99]
    mixture_temps = []

    # iterate thorugh both oil lists and append the approximate value to a new temperature list (mixture_temps)
    for i in range(12):
        approx_val = (oil_temps_a[i] + oil_temps_b[i]) / 2

        # if the approx_val does not exist, pop the last value of the percent yield list
        if np.isnan(approx_val):
            percent_yield.pop(-1)
        else:
            mixture_temps.append(approx_val)
    mixture_tbl = pd.Series(data=mixture_temps, index=percent_yield)
    return get_oil_profile_func(mixture_tbl)



# returns the temperature needed to retrieve the volume given an oil profile function
def oil_snapshot(oil_profile_func, percent):
    try:
        # checks to see if the percentage is within range
        if percent >= 5 and percent <= 99:
            return int(oil_profile_func(percent))
        else:
            return '**Error: Percentage not within range**'
    except:
        print('**Error: parameters are invalid**')
        return []

# returns a the profile table of the mixture, with preset temperatures
def oil_snapshot_tbl(oil_profile_func):
    percent_yield = [5,10,20,30,40,50,60,70,80,90,95,99]
    oil_temps = []
    try:
        for i in range(12):
            oil_temps.append(oil_snapshot(oil_profile_func,percent_yield[i]))
        return pd.Series(data=oil_temps, index=percent_yield)
    except:
        print('**Error: Oil profile function is invalid**')
        return []


[5, 10, 20, 30, 40, 50, 60, 70, 80, 90]
[ 57.55 106.6  155.2  213.2  269.45 323.25 381.75 442.35 515.7  630.35]
278


In [2]:
# prints display and rreturns a graph
def display_graph(oil_profile_func,oil_tbl):
    # prints a matplotlib graph

SyntaxError: unexpected EOF while parsing (<ipython-input-2-be1735f53907>, line 2)

In [ ]:
url_a = \
    'https://www.crudemonitor.ca/crudes/dist.php?acr=MSY&time=recent'
url_b = \
    'https://www.crudemonitor.ca/crudes/dist.php?acr=MSW&time=recent'
oil_temps_a = get_distillation_temps(driver, url_a)
oil_temps_b = get_distillation_temps(driver, url_b)
